# Final Project STA141B: 
## Do public scandals affect the viewer count of Popular Music Artists?

In [1]:
!pip install spotipy

In [2]:
import requests
import pandas as pd
import numpy as np
import sqlalchemy as sqla
# import requests_cache
import plotnine as p9
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
client_id = '65580595dc324a1cb2e890e9800479c4'
client_secret = '20a9994294354eae8ef6cc4ea75e48a3'

client_credentials_manager = SpotifyClientCredentials(client_id,client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [7]:
# extract all features of all (over 100) songs on a playlist into a dataframe
def playlist_to_df(username, playlist_id, tracks_exist = True):
    # get JSON of playlist metadata for first page of songs
    spotipy_response = sp.user_playlist_tracks(username,playlist_id)
    
    if tracks_exist:
        spotipy_response = spotipy_response['tracks']
    
    # record track information
    results = spotipy_response['items']
    
    # while there is a next page of songs (100 songs per page)
    while spotipy_response['next']:
        # replace previous page information with new page
        spotipy_response = sp.next(spotipy_response)
        
        # add track information to list
        results.extend(spotipy_response['items']) 
    
    first = True
    
    # loop through tracks
    for item in results:
        try:
            track = item['track']
            
            # extract id and other features from track metadata and store in dictionary
            ids = track['id']
            
            artist_list = []
            for artist in track['artists']:
                artist_list.append(artist['name'])

            new_row = {'id':[track['id']], 'title':[track['name']], 'all_artists':[artist_list],
                       'popularity':[track['popularity']], 
                       'release_date':[track['album']['release_date']]}

            # extract given features for track and add to dictionary
            features = sp.audio_features(ids)[0]
            
            for key in features:
                new_row[key] = [features[key]]
            
            # turn dictionary into dataframe
            new_df = pd.DataFrame(new_row)
            
            # create new dataframe or add to existing dataframe
            if first:
                features_df = new_df
                first = False
            else:
                features_df = pd.concat([features_df, new_df], ignore_index = True)
                
        except:
            continue
            
    # drop unnecessary features
    features_df = features_df.drop(['speechiness', 'type', 'uri', 'track_href', 'analysis_url'], axis=1)
    
    # return dataframe with all songs and desired features
    return features_df

In [5]:
username = 'spotify:user:ef8zswxc6q41041caiq7x28pa'
playlist_id = 'spotify:playlist:4FLeoROn5GT7n2tZq5XB4V?offset=101'
df_tiktok = playlist_to_df(username, playlist_id)
df_tiktok.to_csv("TikTok_Playlist.csv", index=False)

In [8]:
username = 'spotify:user:spotify'
playlist_id = 'spotify:playlist:37i9dQZF1DWVRSukIED0e9'
df_2019 = playlist_to_df(username, playlist_id, tracks_exist = False)
df_2019.to_csv("Popular_Playlist_2019.csv", index=False)

In [10]:
username = 'spotify:user:spotify'
playlist_id = 'spotify:playlist:37i9dQZF1DX7Jl5KP2eZaS'
df_2020 = playlist_to_df(username, playlist_id, tracks_exist = False)
df_2020.to_csv("Popular_Playlist_2020.csv", index=False)

In [12]:
username = 'spotify:user:nick170708%21'
playlist_id = 'spotify:playlist:5GhQiRkGuqzpWZSE7OU4Se'
df_2021 = playlist_to_df(username, playlist_id, tracks_exist = False)
df_2021.to_csv("Popular_Playlist_2021.csv", index=False)